In [2]:
# decision tree is powerful ML algorithms capable of both regression and classification, which is also foundation for 
# random forest
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
iris = load_iris()

In [9]:
X = iris.data[:, 2:]
y = iris.target

In [11]:
tree_clf = DecisionTreeClassifier(max_depth = 2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [26]:
# we can also use the decision tree to do regression
from sklearn.tree import DecisionTreeRegressor
import numpy as np
tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X, y)
y_prediction = tree_reg.predict(X)

In [27]:
# decision tree instability: decision tree only draws horizontal or vertical decision boundaries. Hence it is not so
# ideal for rotated data set, while we can use PCA to regularize the data space before we apply decision tree
# notice the similiarity between decision tree and binary tree in the traversing complexity log2M. For regression task,
# the number of discrete values is 2^m where m is the number of the depth. sensitivity to variation could be solved by

In [30]:
# plot the regression tree
np.random.seed(42)
m = 200
X = np.random.rand(m,1)
y = 4 *(X-0.5) ** 2
y = y+ np.random.randn(m,1)/10

In [33]:
from sklearn.tree import DecisionTreeRegressor 
tree_reg = DecisionTreeRegressor(max_depth =2, random_state = 42)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [35]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(random_state = 42, max_depth = 2)
tree_reg2 = DecisionTreeRegressor(random_state = 42, max_depth = 3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [36]:
# exercise starting from 7: train and fine tune make_moon dataset

In [38]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples = 10000, noise = 0.4)

In [40]:
# now split train test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.2, random_state = 42)


In [41]:
# use GridsearchCV to fine tune the classifier hyperparameters
from sklearn.model_selection import GridSearchCV

# write params in this way: dictionary, key and corresponding list
params = {'max_leaf_nodes': list(range(2, 1000)),
         'min_samples_split':[2,3,4]}
# define gridsearch model
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state = 42),
                             params, n_jobs = -1, verbose = 2)
# fit the search cv to the data
grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 2994 candidates, totalling 8982 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1992 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 5240 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 8982 out of 8982 | elapsed:   47.2s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 7...6, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [42]:
# we can now use the best parameters to obtain the accuracy 
from sklearn.metrics import accuracy_score
y_prediction = grid_search_cv.predict(X_test)
accuracy_score(y_prediction, y_test)

0.85850000000000004

In [43]:
# accuracy is not so high, this could be due to overfitting during 
# training or other factors

In [44]:
# now we can grow the decision tree to a random forest:
# principle is to train more decision trees which each handles 
# less amount of training data and finally for each data point 
# take the majority vote among all the trees

In [67]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [68]:

from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

TypeError: 'list' object is not callable